In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Let's import the spread measure that you generated before.
spread_meas = pd.read_csv('spread_meas.csv')

In [5]:
prin_comp = pd.read_csv('prin_comp.csv')
del prin_comp['Unnamed: 0']

In [6]:
prin_comp1_rescaled = prin_comp.iloc[:,0] * prin_comp.iloc[:,0].std()\
                      + prin_comp.iloc[:, 0].mean()
prin_comp2_rescaled = prin_comp.iloc[:,1] * prin_comp.iloc[:,1].std()\
                      + prin_comp.iloc[:, 1].mean()
prin_comp_rescaled = pd.concat([prin_comp1_rescaled, 
                                prin_comp2_rescaled],
                               axis=1)
prin_comp_rescaled.head()

,Component 1,Component 2
0,1.763023,1.318156
1,4.831684,2.000901
2,3.607953,1.615184
3,0.959223,0.650714
4,0.834418,0.787859


In [11]:
prin_comp_rescaled = pd.read_csv('prin_comp_rescaled.csv')
del prin_comp_rescaled['Unnamed: 0']
bid_ask = pd.read_csv('bid_ask.csv')

Then, you are ready to calculate the mean value of the principal components to end up with a single variable representing both components.

In [12]:
mean_pca_liq = prin_comp_rescaled.mean(axis=1).mean()
mean_pca_liq

1.0629549058506582

Liqudity adjusted VaR, VaR_Liq, formula is given below. The formula consists of two part:

* VaR and
* Liqudity adjustment

\begin{equation}
\text{VaR_Liq}= \text{VaR} +\text{Liquidity Adjustment}
\end{equation}

\begin{equation}
\text{VaR_Liq}= \text{VaR} + \frac{1}{2}P_{mid}(\mu_{pca}+k\sigma_{pca})
\end{equation}

For the given confidence level of 95\%, standard normal distribution percentile become 1.96, which is represented by k.

So, let's put the pieces together and interpret the result of liqudity adjusted VaR.

In [15]:
#First, let's import the bid-ask spread data that 
#you have produced before.
bid_ask = pd.read_csv('bid_ask_revised.csv')

In [16]:
pca_std = prin_comp_rescaled.mean(axis=1).std()
pm = bid_ask['mid_price'].mean()
k = 1.96

In [17]:
# You know the value of variance-covariance VaR from previous project.
VaR_param = 56073.26

In [18]:
def LVaR (VaR_param, pm, mean_pca_liq, k, pca_std):
    LVaR = VaR_param + (1/2) * pm * (mean_pca_liq + k * pca_std)
    print('The liquidity Adjusted VaR is {}'.format(np.round(LVaR, 2)))

In [19]:
LVaR (VaR_param, pm, mean_pca_liq, k, pca_std)

The liquidity Adjusted VaR is 56185.2


As expected, when liqudity dimension is considered the expected loss turns out to be greater than the palin VaR application. To be more concrete, liquidity adjusted VaR is $\$56185.2$, which is almost $\$1000$ greater than the variance-covariance VaR result, which is $55030.97.

Long story short, taking liquidity into account in estimating the market risk gives us a more realiable and encompassing result.